In [ ]:
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import cupy as cp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from skimage.feature import peak_local_max
from scipy.ndimage import center_of_mass
from scipy.spatial import Delaunay
from collections import Counter
import os

In [ ]:
import sys
sys.path.append('D:/GitHubWSU/pfc-blocks/python/pfc-models/ln-vacancy')

import importlib

import PFC2D_Vacancy as PFC2D_Vacancy
importlib.reload(PFC2D_Vacancy)

In [ ]:
def Plot(sim, file=None):
  fig = go.Figure(data=go.Heatmap(x=np.arange(sim.nx)*sim.dx,
                                  y=np.arange(sim.ny)*sim.dy,
                                  z=sim.phi.get(), colorscale='gray', showscale=False))

  # decide pixels per cell (1 means 1 figure pixel per heatmap cell)
  px_per_cell = 2

  # exact figure pixel size to match heatmap grid
  fig_width = int(sim.nx * px_per_cell)
  fig_height = int(sim.ny * px_per_cell)

  fig.update_layout(
      width=fig_width,
      height=fig_height,
      autosize=False,
      margin=dict(l=0, r=0, t=0, b=0),
      plot_bgcolor='white',
      # place colorbar inside plot or hide it to avoid extra space
      # if you want a colorbar keep it small and place it on top of the image
      # e.g. fig.data[0].colorbar.update({'len':1, 'thickness':8, 'x':0.995})
  )

  fig.update_xaxes(
      showgrid=False, showticklabels=False,
      # force axis to use full horizontal domain
      domain=[0.0, 1.0],
      range=[0, sim.nx * sim.dx]   # match the heatmap coordinates
  )

  fig.update_yaxes(
      showgrid=False, showticklabels=False,
      domain=[0.0, 1.0],
      range=[0, sim.ny * sim.dy],  # match the heatmap coordinates
      scaleanchor='x',             # lock aspect ratio so a unit in x equals unit in y
      scaleratio=1
  )

  # optionally hide the colorbar to remove its margin impact
  if hasattr(fig.data[0], 'colorbar'):
      fig.data[0].colorbar.update({'thickness': 8, 'len': 1, 'x': 0.995})  # place thin bar on right
      # or remove it: fig.data[0].colorbar = dict() or fig.data[0].showscale = False

  fig.show()

  if file is not None:
    fig.write_image(file)

## 1. Standard PFC

In [ ]:
n0 = -0.62

In [ ]:
beta_val = 0.25 # 1.0
epsilon_val = 0.35 #-2.82 -0.1
g_val = 1.2 #-1
# phi_s = -0.35 #0.54  -0.1 #-0.18 #0.5 #
# phi_l = -0.60 #0.32676978996474756
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim_std = PFC2D_Vacancy.PFC2D_Vacancy()
sim_std.minLog = -100.0
sim_std.minPhi = np.exp(sim_std.minLog)
sim_std.phiMax = 50
sim_std.parms.epsilon = epsilon_val
sim_std.parms.a = 0
sim_std.parms.beta = beta_val
sim_std.parms.b = (0, -0.2, 0.0)
sim_std.parms.q = (1, 3**0.5, 2)
sim_std.parms.g = g_val
sim_std.parms.v0 = 1.0

sim_std.parms.Hng = 0
sim_std.parms.Hln = 0
sim_std.parms.N = 288
sim_std.parms.Nx = 288
sim_std.parms.Ny = 288
sim_std.parms.PPU = 20
sim_std.parms.scaleFactor = 1/q_eff
sim_std.parms.eta = 0.15 #0.0025
sim_std.parms.dt = 1e-2
sim_std.parms.seed = 34125
sim_std.parms.NoiseDynamicsFlag = False
sim_std.parms.NoiseTimeSmoothFlag = True
sim_std.parms.NoiseTimeSmoothingFrames = 0
sim_std.parms.Noise_CutoffOmega = 0.04

sim_std.parms.phi0 = n0
sim_std.InitParms()
# sim_std.SetGeometry(288, 288, 15, scalefactor = 1/q_eff)
sim_std.parms.Noise_CutoffK = 2*np.pi/(sim_std.dx*sim_std.nx) * sim_std.mx

# sim_std.InitFieldCrystal(A = 1.65*np.sign(sim_std.parms.phi0) * sim_std.parms.phi0/3, noisy=False, scalefactor = 1/q_eff)
sim_std.InitFieldFlat(noisy=True)
# sim_std.phi = cp.sin(sim_std.x)
# sim_std.phi[144-20:144+20,144:144+40] = phi_l
# sim_std.phi[0:64,:] = phi_l
# sim_std.phi[-64:,:] = phi_l
# sim_std.phi[:,144-72:144+72] = sim_std.phi[144-72:144+72,:].T
sim_std.t = 0

Plot(sim_std, )#"./out/prospectus/fig-sl-coexistence-stdpfc-t0.png")
phi_std0 = sim_std.phi.get()

# Total steps and checkpoints
total_steps = int(1e3)
energy_calcs = 10

sim_std.CalcEnergyDensity()

# Run sim_stdulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_std.CalcEnergyDensity()
      print(f'f: {sim_std.f:.10f}, t: {sim_std.t:.2f}, min: {sim_std.phi.min()}, max: {sim_std.phi.max()}, mean: {sim_std.phi.mean()}')

    # Step sim_stdulation
    sim_std.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim_std, )#"./out/prospectus/fig-sl-coexistence-stdpfc-t10.png")
phi_std1 = sim_std.phi.get()

# Total steps and checkpoints
sim_std.SetDT(1e-1)
total_steps = int(1e4)
energy_calcs = 10

sim_std.CalcEnergyDensity()

# Run sim_stdulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_std.CalcEnergyDensity()
      print(f'f: {sim_std.f:.10f}, t: {sim_std.t:.2f}, min: {sim_std.phi.min()}, max: {sim_std.phi.max()}, mean: {sim_std.phi.mean()}')

    # Step sim_stdulation
    sim_std.TimeStepCross()
  except Exception as e:
    print(e)
    break

total_steps = int(3e4)
sim_std.SetDT(sim_std.parms.dt * 0.1)
energy_calcs = 3

# Run sim_stdulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_std.CalcEnergyDensity()
      print(f'f: {sim_std.f:.10f}, t: {sim_std.t:.2f}, min: {sim_std.phi.min()}, max: {sim_std.phi.max()}, mean: {sim_std.phi.mean()}')
      sim_std.SetDT(sim_std.parms.dt * 0.1)

    # Step sim_stdulation
    sim_std.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim_std, )#"./out/prospectus/fig-sl-coexistence-stdpfc-t1000.png")
phi_std = sim_std.phi.get()

# fig = go.Figure()
# fig.add_trace(go.Scatter(x = sim_std.y[:,0].get()/a, y=phi_std0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
# fig.add_trace(go.Scatter(x = sim_std.y[:,0].get()/a, y=phi_std1.mean(axis=1), mode='lines', name='t=10', line_color='skyblue', opacity=0.75))
# fig.add_trace(go.Scatter(x = sim_std.y[:,0].get()/a, y=phi_std.mean(axis=1), mode='lines', name='t=1000', line_color='orange', opacity=0.99))

# fig.update_layout(
#   height=300,
#   width=800,
#   plot_bgcolor='white',          # plot area background
#   paper_bgcolor='white',         # surrounding page background
#   margin=dict(l=0, r=0, t=0, b=0),
#   legend=dict(
#       x=0.02,               # horizontal position (0 = left, 1 = right)
#       y=0.98,               # vertical position (0 = bottom, 1 = top)
#       xanchor='left',       # anchor legend's x to its left side
#       yanchor='top',        # anchor legend's y to its top
#       bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
#       bordercolor='black',
#       borderwidth=1,
#       orientation='v'       # 'v' for vertical, 'h' for horizontal
#   ),
# )
# light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

# fig.update_xaxes(
#     showgrid=True, gridcolor=light_gray, gridwidth=1,
#     zeroline=True, zerolinecolor=light_gray,
#     dtick=1,
#     linecolor='black', mirror=True, ticks='outside'
# )
# fig.update_yaxes(
#     showgrid=True, gridcolor=light_gray, gridwidth=1,
#     zeroline=True, zerolinecolor=light_gray,
#     dtick=0.05,
#     linecolor='black', mirror=True, ticks='outside'
# )
# fig.show()
# # fig.write_image("./out/prospectus/fig-sl-coexistence-stdpfc-profile.png")

## 2. Log PFC

In [ ]:
# epsilon 0.5
phi_s, phi_l = 0.80, 0.54
phi_s, phi_l = 0.75, 0.50
phi_s, phi_l = 0.73, 0.50

# epsilon 0.88
phi_s, phi_l, amp = 0.84, 0.76, 0.30 # amp too high
phi_s, phi_l, amp = 0.84, 0.76, 0.25 # amp too low
phi_s, phi_l, amp = 0.84, 0.76, 0.27 # amp correct - first atom amp is low
phi_s, phi_l, amp = 0.84, 0.80, 0.27 # amp a little high - first atom a little low
phi_s, phi_l, amp = 0.84, 0.805, 0.265 # increased run time, appeas that crystal is growing

phi_s, phi_l, amp = 0.84, 0.76, 0.265 # appears crystal is shrinking
phi_s, phi_l, amp = 0.90, 0.76, 0.265 # appears crystal is shrinking

# g = -1.0, epsilon = -1.6
phi_s, phi_l, amp = 0.555, 0.40, 0.4

In [ ]:
beta_val = 0.289 # 1.0
epsilon_val = -1.6 #-2.82 -0.1
g_val = -1.0
# phi_l = 0.54
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim_log = PFC2D_Vacancy.PFC2D_Vacancy()
sim_log.minLog = -100.0
sim_log.minPhi = np.exp(sim_log.minLog)
sim_log.phiMax = 50
sim_log.parms.epsilon = epsilon_val
sim_log.parms.a = 0
sim_log.parms.beta = beta_val
sim_log.parms.b = (0, -0.2, 0.0)
sim_log.parms.q = (1, 3**0.5, 2)
sim_log.parms.g = g_val
sim_log.parms.v0 = 1.0

sim_log.parms.Hng = 0
sim_log.parms.Hln = 1
sim_log.parms.N = 288
sim_log.parms.Nx = 288
sim_log.parms.Ny = 288
sim_log.parms.PPU = 20
sim_log.parms.scaleFactor = 1/q_eff
sim_log.parms.eta = 0.1 #0.0025
sim_log.parms.dt = 1e-3
sim_log.parms.seed = 34125
sim_log.parms.NoiseDynamicsFlag = False
sim_log.parms.NoiseTimeSmoothFlag = True
sim_log.parms.NoiseTimeSmoothingFrames = 0
sim_log.parms.Noise_CutoffOmega = 0.04

sim_log.parms.phi0 = phi_s
sim_log.InitParms()
# sim_log.SetGeometry(288, 288, 15, scalefactor = 1/q_eff)
sim_log.parms.Noise_CutoffK = 2*np.pi/(sim_log.dx*sim_log.nx) * sim_log.mx

# sim_log.InitFieldCrystal(A = amp * np.sign(sim_log.parms.phi0) * sim_log.parms.phi0/3, noisy=False, scalefactor = 1/q_eff)
sim_log.InitFieldFlat(noisy=True)
# sim_log.phi = cp.sin(sim_log.x)
# sim_log.phi[144-20:144+20,144:144+40] = phi_l
# sim_log.phi[0:64,:] = phi_l
# sim_log.phi[-64:,:] = phi_l
# sim_log.phi[:,144-72:144+72] = sim_log.phi[144-72:144+72,:].T
sim_log.t = 0

Plot(sim_log)#, "./out/prospectus/fig-sl-coexistence-logpfc-t0.png")
phi_log0 = sim_log.phi.get()

# Total steps and checkpoints
total_steps = int(1e3)
energy_calcs = 10

sim_log.CalcEnergyDensity()

# Run sim_logulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_log.CalcEnergyDensity()
      print(f'f: {sim_log.f:.10f}, t: {sim_log.t:.2f}, min: {sim_log.phi.min()}, max: {sim_log.phi.max()}, mean: {sim_log.phi.mean()}')

    # Step sim_logulation
    sim_log.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim_log)#, "./out/prospectus/fig-sl-coexistence-logpfc-t1.png")
phi_log1 = sim_log.phi.get()

# Total steps and checkpoints
sim_log.SetDT(1e-2)
total_steps = int(3e4)
energy_calcs = 10

sim_log.CalcEnergyDensity()

# Run sim_logulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_log.CalcEnergyDensity()
      print(f'f: {sim_log.f:.10f}, t: {sim_log.t:.2f}, min: {sim_log.phi.min()}, max: {sim_log.phi.max()}, mean: {sim_log.phi.mean()}')

    # Step sim_logulation
    sim_log.TimeStepCross()
  except Exception as e:
    print(e)
    break

total_steps = int(1e4)
sim_log.SetDT(sim_log.parms.dt * 0.1)
energy_calcs = 3

# Run sim_logulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_log.CalcEnergyDensity()
      print(f'f: {sim_log.f:.10f}, t: {sim_log.t:.2f}, min: {sim_log.phi.min()}, max: {sim_log.phi.max()}, mean: {sim_log.phi.mean()}')
      sim_log.SetDT(sim_log.parms.dt * 0.1)

    # Step sim_logulation
    sim_log.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim_log)#, "./out/prospectus/fig-sl-coexistence-logpfc-t100.png")
phi_log = sim_log.phi.get()

# fig = go.Figure()
# fig.add_trace(go.Scatter(x = sim_log.y[:,0].get()/a, y=phi_log0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
# fig.add_trace(go.Scatter(x = sim_log.y[:,0].get()/a, y=phi_log1.mean(axis=1), mode='lines', name='t=1', line_color='skyblue', opacity=0.75))
# fig.add_trace(go.Scatter(x = sim_log.y[:,0].get()/a, y=phi_log.mean(axis=1), mode='lines', name='t=100', line_color='orange', opacity=0.99))

# fig.update_layout(
#   height=300,
#   width=800,
#   plot_bgcolor='white',          # plot area background
#   paper_bgcolor='white',         # surrounding page background
#   margin=dict(l=0, r=0, t=0, b=0),
#   legend=dict(
#       x=0.02,               # horizontal position (0 = left, 1 = right)
#       y=0.98,               # vertical position (0 = bottom, 1 = top)
#       xanchor='left',       # anchor legend's x to its left side
#       yanchor='top',        # anchor legend's y to its top
#       bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
#       bordercolor='black',
#       borderwidth=1,
#       orientation='v'       # 'v' for vertical, 'h' for horizontal
#   ),
# )
# light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

# fig.update_xaxes(
#     showgrid=True, gridcolor=light_gray, gridwidth=1,
#     zeroline=True, zerolinecolor=light_gray,
#     dtick=1,
#     linecolor='black', mirror=True, ticks='outside'
# )
# fig.update_yaxes(
#     showgrid=True, gridcolor=light_gray, gridwidth=1,
#     zeroline=True, zerolinecolor=light_gray,
#     dtick=0.1,
#     linecolor='black', mirror=True, ticks='outside'
# )
# fig.show()
# # fig.write_image("./out/prospectus/fig-sl-coexistence-logpfc-profile.png")

In [ ]:
sim.phi[50:100, 50:100].mean(), sim.phi[50:102, 50:105].mean(), sim.phi[47:86, 53:95].mean()

In [ ]:
beta_val = 0.289 # 1.0
epsilon_val = -1.8 #-2.82 -0.1
g_val = -1.0
phi_s = 0.48  #-0.1 #-0.18 #0.5 #
# phi_l = 0.54
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim2 = PFC2D_Vacancy.PFC2D_Vacancy()
sim2.minLog = -100.0
sim2.minPhi = np.exp(sim2.minLog)
sim2.phiMax = 50
sim2.parms.epsilon = epsilon_val
sim2.parms.a = 0
sim2.parms.beta = beta_val
sim2.parms.b = (0, -0.2, 0.0)
sim2.parms.q = (1, 3**0.5, 2)
sim2.parms.g = g_val
sim2.parms.v0 = 1.0
sim2.parms.phi0 = phi_s

sim2.parms.Hng = 0
sim2.parms.Hln = 1
sim2.parms.N = 288
sim2.parms.Nx = 288
sim2.parms.Ny = 288
sim2.parms.PPU = 20
sim2.parms.scaleFactor = 1/q_eff
sim2.parms.eta = 0.1 #0.0025
sim2.parms.dt = 1e-3
sim2.parms.seed = 34125
sim2.parms.NoiseDynamicsFlag = False
sim2.parms.NoiseTimeSmoothFlag = True
sim2.parms.NoiseTimeSmoothingFrames = 0
sim2.parms.Noise_CutoffOmega = 0.04

sim2.parms.phi0 = phi_s
sim2.InitParms()
# sim2.SetGeometry(288, 288, 15, scalefactor = 1/q_eff)
sim2.parms.Noise_CutoffK = 2*np.pi/(sim2.dx*sim2.nx) * sim2.mx

# sim2.InitFieldCrystal(A = amp * np.sign(sim2.parms.phi0) * sim2.parms.phi0/3, noisy=False, scalefactor = 1/q_eff)
sim2.InitFieldFlat(noisy=True)
# sim2.phi = cp.sin(sim2.x)
# sim2.phi[144-20:144+20,144:144+40] = phi_l
# sim2.phi[0:64,:] = phi_l
# sim2.phi[-64:,:] = phi_l
# sim2.phi[:,144-72:144+72] = sim2.phi[144-72:144+72,:].T
sim2.t = 0

Plot(sim2)#, "./out/prospectus/fig-sl-coexistence-logpfc-t0.png")
phi_log0 = sim2.phi.get()

# Total steps and checkpoints
total_steps = int(1e3)
energy_calcs = 10

sim2.CalcEnergyDensity()

# Run sim2ulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim2.CalcEnergyDensity()
      print(f'f: {sim2.f:.10f}, t: {sim2.t:.2f}, min: {sim2.phi.min()}, max: {sim2.phi.max()}, mean: {sim2.phi.mean()}')

    # Step sim2ulation
    sim2.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim2)#, "./out/prospectus/fig-sl-coexistence-logpfc-t1.png")
phi_log1 = sim2.phi.get()

# Total steps and checkpoints
sim2.SetDT(1e-2)
total_steps = int(3e4)
energy_calcs = 10

sim2.CalcEnergyDensity()

# Run sim2ulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim2.CalcEnergyDensity()
      print(f'f: {sim2.f:.10f}, t: {sim2.t:.2f}, min: {sim2.phi.min()}, max: {sim2.phi.max()}, mean: {sim2.phi.mean()}')

    # Step sim2ulation
    sim2.TimeStepCross()
  except Exception as e:
    print(e)
    break

total_steps = int(1e4)
sim2.SetDT(sim2.parms.dt * 0.1)
energy_calcs = 3

# Run sim2ulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim2.CalcEnergyDensity()
      print(f'f: {sim2.f:.10f}, t: {sim2.t:.2f}, min: {sim2.phi.min()}, max: {sim2.phi.max()}, mean: {sim2.phi.mean()}')
      sim2.SetDT(sim2.parms.dt * 0.1)

    # Step sim2ulation
    sim2.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim2)#, "./out/prospectus/fig-sl-coexistence-logpfc-t100.png")
phi_log = sim2.phi.get()

# fig = go.Figure()
# fig.add_trace(go.Scatter(x = sim2.y[:,0].get()/a, y=phi_log0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
# fig.add_trace(go.Scatter(x = sim2.y[:,0].get()/a, y=phi_log1.mean(axis=1), mode='lines', name='t=1', line_color='skyblue', opacity=0.75))
# fig.add_trace(go.Scatter(x = sim2.y[:,0].get()/a, y=phi_log.mean(axis=1), mode='lines', name='t=100', line_color='orange', opacity=0.99))

# fig.update_layout(
#   height=300,
#   width=800,
#   plot_bgcolor='white',          # plot area background
#   paper_bgcolor='white',         # surrounding page background
#   margin=dict(l=0, r=0, t=0, b=0),
#   legend=dict(
#       x=0.02,               # horizontal position (0 = left, 1 = right)
#       y=0.98,               # vertical position (0 = bottom, 1 = top)
#       xanchor='left',       # anchor legend's x to its left side
#       yanchor='top',        # anchor legend's y to its top
#       bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
#       bordercolor='black',
#       borderwidth=1,
#       orientation='v'       # 'v' for vertical, 'h' for horizontal
#   ),
# )
# light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

# fig.update_xaxes(
#     showgrid=True, gridcolor=light_gray, gridwidth=1,
#     zeroline=True, zerolinecolor=light_gray,
#     dtick=1,
#     linecolor='black', mirror=True, ticks='outside'
# )
# fig.update_yaxes(
#     showgrid=True, gridcolor=light_gray, gridwidth=1,
#     zeroline=True, zerolinecolor=light_gray,
#     dtick=0.1,
#     linecolor='black', mirror=True, ticks='outside'
# )
# fig.show()
# # fig.write_image("./out/prospectus/fig-sl-coexistence-logpfc-profile.png")

In [ ]:
sim2.phi[10:100,-50:-10].mean()

## Profile plots

In [ ]:
sim.mx, sim.nx, sim.dx, sim.nx*sim.dx/sim.mx
a = sim.nx*sim.dx/sim.mx

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std1.mean(axis=1), mode='lines', name='t=10', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_std.mean(axis=1), mode='lines', name='t=1000', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.05,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
# fig.write_image("./out/prospectus/fig-sl-coexistence-stdpfc-profile.png")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log0.mean(axis=1), mode='lines', name='t=0', line_color='limegreen', opacity=0.5))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log1.mean(axis=1), mode='lines', name='t=1', line_color='skyblue', opacity=0.75))
fig.add_trace(go.Scatter(x = sim.y[:,0].get()/a, y=phi_log.mean(axis=1), mode='lines', name='t=100', line_color='orange', opacity=0.99))

fig.update_layout(
  height=300,
  width=800,
  plot_bgcolor='white',          # plot area background
  paper_bgcolor='white',         # surrounding page background
  margin=dict(l=0, r=0, t=0, b=0),
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)
light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)

fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    dtick=0.1,
    linecolor='black', mirror=True, ticks='outside'
)
fig.show()
# fig.write_image("./out/prospectus/fig-sl-coexistence-logpfc-profile.png")